In [1]:
# Import Packages
import numpy as np
import scipy.stats as stats
import os
import scipy.io as scio
from ncdump_python3 import ncdump
import pickle
import Jacks_Functions as JF
import Area_Avg
import matplotlib
from matplotlib import pyplot as plt
from scipy.interpolate import griddata
from netCDF4 import Dataset
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

In [2]:
#Load in Dimensions
LLL = Dataset('LatLon.nc')
WACCM4_Lat = np.squeeze(LLL.variables['lat'])
WACCM4_Lon = np.squeeze(LLL.variables['lon'])
LLL.close()

Source_File = Dataset('Kernels/Interpolated/CAM3_Kernels.nc')

CAM3_Lat = np.squeeze(Source_File.variables['lat'])
CAM3_Lon = np.squeeze(Source_File.variables['lon'])

cx,cy = np.meshgrid(CAM3_Lon,CAM3_Lat)
wx,wy = np.meshgrid(WACCM4_Lon,WACCM4_Lat)

wAVD_PS = pickle.load(open('wAVD_Base_PS.pickle','rb'))
#interpolate the surface pressure to the CAM3 grid
wAVD_PS_interp = np.zeros([12,64,128])
for i in range(12):
    wAVD_PS_interp[i,:,:] = griddata(\
    (wx.flatten(),wy.flatten()),wAVD_PS[i,:,:].flatten(),\
    (cx,cy),method='linear')/100 #swap to hpa in the process
    
#Lists for iterating
Months = np.arange(0,12)
FP_Ensemble_Members = np.arange(5)

In [4]:
#Tile the plevs to match the shape of the kernel
CMIP_plevs_scalar = np.squeeze(Source_File.variables['lev'])

A = np.tile(CMIP_plevs_scalar.T,(CAM3_Lon.size,1))
B = np.transpose(A[:,:,None],(0,1,2))
C = np.tile(B,(1,1,CAM3_Lat.size))
D = np.transpose(C[:,:,:,None],(0,1,2,3))
E = np.tile(D,(1,1,1,12))
CMIP_plevs = np.swapaxes(E,3,0)
CMIP_plevs.shape

(12, 17, 64, 128)

In [7]:
#load base/response variables for saturation mixing ratio calculations
wAVD_Q_Base = np.swapaxes(pickle.load(open(\
   "Vars for Specific Humidity Calculations/wAVD_Q_2006-2015_CAM3Grid.pickle","rb")\
                                       ,encoding='latin1'),3,4)

wAVD_Q_Response = np.swapaxes(pickle.load(open(\
   "Vars for Specific Humidity Calculations/wAVD_Q_2056-2065_CAM3Grid.pickle","rb")\
                                       ,encoding='latin1'),3,4)

wAVD_Ta_Base = np.swapaxes(pickle.load(open(\
   "Vars for Specific Humidity Calculations/wAVD_Ta_2006-2015_CAM3Grid.pickle","rb")\
                                       ,encoding='latin1'),3,4)

wAVD_Ta_Response = np.swapaxes(pickle.load(open(\
   "Vars for Specific Humidity Calculations/wAVD_Ta_2056-2065_CAM3Grid.pickle","rb")\
                                       ,encoding='latin1'),3,4)

In [8]:
wAVD_sHum_Base = np.zeros([5,12,17,64,128])
wAVD_sHum_Response = np.zeros([5,12,17,64,128])

for i in range(FP_Ensemble_Members.size):
    #saturation mixing ratio for the baseline period
    wAVD_sHum_Base[i,:,:,:,:] = JF.Calc_SatSpec_Hum(\
    Ta=wAVD_Ta_Base[i,:,:,:,:],P = CMIP_plevs)/1000 #in kg/kg
    
    #saturation mixing ratio for the response period
    wAVD_sHum_Response[i,:,:,:,:] = JF.Calc_SatSpec_Hum(\
    Ta=wAVD_Ta_Response[i,:,:,:,:],P = CMIP_plevs)/1000 #in kg/kg
    
    
#relative humidity of the baseline, in kg/kg
wAVD_rh = wAVD_Q_Base/wAVD_sHum_Base
    
#new change in saturation spec hum under a +1k climate
wAVD_dQs1k = (wAVD_sHum_Response-wAVD_sHum_Base)/(wAVD_Ta_Response-wAVD_Ta_Base)

#change in spec hum assuming relative humidity remains constant
wAVD_dQ1k = wAVD_dQs1k*wAVD_rh

#logarithm of spec humidity required to increase temp by 1K under constant relative humidity
wAVD_dlnQ1k = wAVD_dQ1k/wAVD_Q_Base

In [9]:
#water vapour mixing ratios and temperature responses

#surface temperature in K
wAVD_dST_CAM3Grid = np.swapaxes(pickle.load(open(\
   "Future Projection CC Responses/wAVD_dST_CAM3Grid.pickle","rb"),encoding='latin1'),2,3)

#dln(specific humidity) in kg/kg
wAVD_dlnQ_CAM3Grid = np.swapaxes(pickle.load(open(\
   "Future Projection CC Responses/wAVD_dlnQ_CAM3Grid.pickle","rb"),encoding='latin1'),3,4)

#normalize the dln(specific humidity) by the amount required to increase temperature by 1K
    #(by the same anomaly used to compute the kernel)
wAVD_dlnQ_CAM3Grid_Norm = wAVD_dlnQ_CAM3Grid/wAVD_dlnQ1k

In [10]:
#Calculate pressure level thickness using CMIP pressure levels and surface pressure
dp = np.zeros([12,17,64,128])

for i in range(12):
    for j in range(64):
        for k in range(128):
            dp[i,:,j,k] = JF.PlevThck(\
            PS = wAVD_PS_interp[i,j,k],plevs=CMIP_plevs_scalar,\
            p_top = min(CMIP_plevs_scalar))
            
dp = np.ma.masked_invalid(dp[:,:,:,:]/100) #CAM3 Kernel units are scaled per 100 mb

In [11]:
#estimation of the tropopause. 300hPA at the poles, raising
    #linearly to 100hpa at the tropics

p_tropopause_zonalmean_linear = np.zeros([64])

for y in range(64):
    if y <= len(CAM3_Lat)/2:
        if y == 0:
            p_tropopause_zonalmean_linear[y] = 300
        elif y == len(CAM3_Lat)/2:
            p_tropopause_zonalmean_linear[y] = 100
        else:
            p_tropopause_zonalmean_linear[y] = \
            p_tropopause_zonalmean_linear[y-1]\
            -(200/(len(CAM3_Lat)/2-1))
    else:
        if y == len(CAM3_Lat)/2:
            p_tropopause_zonalmean_linear[y] = 100
        elif y == len(CAM3_Lat):
            p_tropopause_zonalmean_linear[y] = 300
        else:
            p_tropopause_zonalmean_linear[y] = \
            p_tropopause_zonalmean_linear[y-1]\
            +(200/(len(CAM3_Lat)/2-1))
            
Q = np.tile(p_tropopause_zonalmean_linear.T,(CAM3_Lon.size,1))
R = np.transpose(Q[:,:,None],(0,1,2))
S = np.tile(R,(1,1,17))
T = np.transpose(S[:,:,:,None],(0,1,2,3))
U = np.tile(T,(1,1,1,12))
V = np.swapaxes(U,3,0)
p_tropopause_CAM3 = np.swapaxes(V,2,1)

#how does it look?
p_tropopause_CAM3.shape

(12, 17, 64, 128)

In [25]:
#CAM3 Kernels
Kernel_CAM3_LW_Q = np.ma.masked_greater(np.squeeze(Source_File.variables['WVlw_TOA']),1e10)
Kernel_CAM3_SW_Q = np.ma.masked_greater(np.squeeze(Source_File.variables['WVsw_TOA']),1e10)

In [18]:
CAM3_wAVD_dlnQ_tropo = np.zeros([FP_Ensemble_Members.size,12,17,64,128])
CAM3_wAVD_dLW_WV = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_wAVD_dSW_WV = np.zeros([FP_Ensemble_Members.size,12,64,128])
CAM3_wAVD_net_WV = np.zeros([FP_Ensemble_Members.size,12,64,128])

for i in range(FP_Ensemble_Members.size):
    
    #mask out the stratosphere
    CAM3_wAVD_dlnQ_tropo[i,:,:,:,:] = \
    wAVD_dlnQ_CAM3Grid_Norm[i,:,:,:,:]*(CMIP_plevs>=p_tropopause_CAM3)
    
    #multiply kernels and vertically integrate
    CAM3_wAVD_dLW_WV[i,:,:,:] = \
    np.ma.sum(CAM3_wAVD_dlnQ_tropo[i,:,:,:,:]*Kernel_CAM3_LW_Q*dp,axis=1)
    
    CAM3_wAVD_dSW_WV[i,:,:,:] = \
    np.ma.sum(CAM3_wAVD_dlnQ_tropo[i,:,:,:,:]*Kernel_CAM3_SW_Q*dp,axis=1)
    
    #add shortwave and longwave responses
    CAM3_wAVD_net_WV[i,:,:,:] = CAM3_wAVD_dLW_WV[i,:,:,:]+CAM3_wAVD_dSW_WV[i,:,:,:]

In [19]:
CAM3_wAVD_WV_EnergyB_AA = np.zeros([FP_Ensemble_Members.size,12])
CAM3_wAVD_WV_EnergyB_GA = np.zeros([FP_Ensemble_Members.size,12])

for i in range(FP_Ensemble_Members.size):
    CAM3_wAVD_WV_EnergyB_AA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_wAVD_net_WV[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    CAM3_wAVD_WV_EnergyB_GA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_wAVD_net_WV[i,:,:,:],CAM3_Lat,CAM3_Lon)

In [20]:
print(np.mean(CAM3_wAVD_WV_EnergyB_AA,axis=1))
print(np.mean(CAM3_wAVD_WV_EnergyB_GA,axis=1))

[1.55098349 1.4764823  1.4654365  1.47971293 1.52863169]
[2.59172535 2.6525966  2.75870729 2.73191226 2.5594497 ]


In [21]:
wAVD_dST_CAM3Grid_AA = np.zeros([5,12])
wAVD_dST_CAM3Grid_GA = np.zeros([5,12])

for i in range(FP_Ensemble_Members.size):
    wAVD_dST_CAM3Grid_AA[i,:] = Area_Avg.LatLonavg_Time(\
    wAVD_dST_CAM3Grid[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    wAVD_dST_CAM3Grid_GA[i,:] = Area_Avg.LatLonavg_Time(\
    wAVD_dST_CAM3Grid[i,:,:,:],CAM3_Lat,CAM3_Lon)

In [22]:
CAM3_wAVD_WV_Feedback_AA = CAM3_wAVD_WV_EnergyB_AA/wAVD_dST_CAM3Grid_AA
CAM3_wAVD_WV_Feedback_GA = CAM3_wAVD_WV_EnergyB_GA/wAVD_dST_CAM3Grid_GA

In [23]:
CAM3_wAVD_WV_EnergyB_Annual_AA = np.mean(CAM3_wAVD_WV_EnergyB_AA,axis=1)
CAM3_wAVD_WV_EnergyB_SON_AA = np.mean(CAM3_wAVD_WV_EnergyB_AA[:,8:11],axis=1)
CAM3_wAVD_WV_EnergyB_JJA_AA = np.mean(CAM3_wAVD_WV_EnergyB_AA[:,5:8],axis=1)
CAM3_wAVD_WV_EnergyB_MAM_AA = np.mean(CAM3_wAVD_WV_EnergyB_AA[:,2:5],axis=1)
CAM3_wAVD_WV_EnergyB_DJF_AA = np.mean((CAM3_wAVD_WV_EnergyB_AA[:,0],
                                       CAM3_wAVD_WV_EnergyB_AA[:,1],\
                                       CAM3_wAVD_WV_EnergyB_AA[:,11]),axis=0)

CAM3_wAVD_WV_EnergyB_Annual_GA = np.mean(CAM3_wAVD_WV_EnergyB_GA,axis=1)
CAM3_wAVD_WV_EnergyB_SON_GA = np.mean(CAM3_wAVD_WV_EnergyB_GA[:,8:11],axis=1)
CAM3_wAVD_WV_EnergyB_JJA_GA = np.mean(CAM3_wAVD_WV_EnergyB_GA[:,5:8],axis=1)
CAM3_wAVD_WV_EnergyB_MAM_GA = np.mean(CAM3_wAVD_WV_EnergyB_GA[:,2:5],axis=1)
CAM3_wAVD_WV_EnergyB_DJF_GA = np.mean((CAM3_wAVD_WV_EnergyB_GA[:,0],
                                       CAM3_wAVD_WV_EnergyB_GA[:,1],\
                                       CAM3_wAVD_WV_EnergyB_GA[:,11]),axis=0)

In [24]:
CAM3_wAVD_WV_Feedback_Annual_AA = np.mean(CAM3_wAVD_WV_Feedback_AA,axis=1)
CAM3_wAVD_WV_Feedback_SON_AA = np.mean(CAM3_wAVD_WV_Feedback_AA[:,8:11],axis=1)
CAM3_wAVD_WV_Feedback_JJA_AA = np.mean(CAM3_wAVD_WV_Feedback_AA[:,5:8],axis=1)
CAM3_wAVD_WV_Feedback_MAM_AA = np.mean(CAM3_wAVD_WV_Feedback_AA[:,2:5],axis=1)
CAM3_wAVD_WV_Feedback_DJF_AA = np.mean((CAM3_wAVD_WV_Feedback_AA[:,0],
                                       CAM3_wAVD_WV_Feedback_AA[:,1],\
                                       CAM3_wAVD_WV_Feedback_AA[:,11]),axis=0)

CAM3_wAVD_WV_Feedback_Annual_GA = np.mean(CAM3_wAVD_WV_Feedback_GA,axis=1)
CAM3_wAVD_WV_Feedback_SON_GA = np.mean(CAM3_wAVD_WV_Feedback_GA[:,8:11],axis=1)
CAM3_wAVD_WV_Feedback_JJA_GA = np.mean(CAM3_wAVD_WV_Feedback_GA[:,5:8],axis=1)
CAM3_wAVD_WV_Feedback_MAM_GA = np.mean(CAM3_wAVD_WV_Feedback_GA[:,2:5],axis=1)
CAM3_wAVD_WV_Feedback_DJF_GA = np.mean((CAM3_wAVD_WV_Feedback_GA[:,0],
                                       CAM3_wAVD_WV_Feedback_GA[:,1],\
                                       CAM3_wAVD_WV_Feedback_GA[:,11]),axis=0)

In [38]:
CAM3_wAVD_WV_EnergyB_Annual_AA_file = open("CAM3_wAVD_WV_EnergyB_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_Annual_AA,CAM3_wAVD_WV_EnergyB_Annual_AA_file)
CAM3_wAVD_WV_EnergyB_Annual_AA_file.close()

CAM3_wAVD_WV_EnergyB_SON_AA_file = open("CAM3_wAVD_WV_EnergyB_SON_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_SON_AA,CAM3_wAVD_WV_EnergyB_SON_AA_file)
CAM3_wAVD_WV_EnergyB_SON_AA_file.close()

CAM3_wAVD_WV_EnergyB_JJA_AA_file = open("CAM3_wAVD_WV_EnergyB_JJA_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_JJA_AA,CAM3_wAVD_WV_EnergyB_JJA_AA_file)
CAM3_wAVD_WV_EnergyB_JJA_AA_file.close()

CAM3_wAVD_WV_EnergyB_MAM_AA_file = open("CAM3_wAVD_WV_EnergyB_MAM_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_MAM_AA,CAM3_wAVD_WV_EnergyB_MAM_AA_file)
CAM3_wAVD_WV_EnergyB_MAM_AA_file.close()

CAM3_wAVD_WV_EnergyB_DJF_AA_file = open("CAM3_wAVD_WV_EnergyB_DJF_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_DJF_AA,CAM3_wAVD_WV_EnergyB_DJF_AA_file)
CAM3_wAVD_WV_EnergyB_DJF_AA_file.close()

CAM3_wAVD_WV_Feedback_Annual_AA_file = open("CAM3_wAVD_WV_Feedback_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_Annual_AA,CAM3_wAVD_WV_Feedback_Annual_AA_file)
CAM3_wAVD_WV_Feedback_Annual_AA_file.close()

CAM3_wAVD_WV_Feedback_SON_AA_file = open("CAM3_wAVD_WV_Feedback_SON_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_SON_AA,CAM3_wAVD_WV_Feedback_SON_AA_file)
CAM3_wAVD_WV_Feedback_SON_AA_file.close()

CAM3_wAVD_WV_Feedback_JJA_AA_file = open("CAM3_wAVD_WV_Feedback_JJA_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_JJA_AA,CAM3_wAVD_WV_Feedback_JJA_AA_file)
CAM3_wAVD_WV_Feedback_JJA_AA_file.close()

CAM3_wAVD_WV_Feedback_MAM_AA_file = open("CAM3_wAVD_WV_Feedback_MAM_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_MAM_AA,CAM3_wAVD_WV_Feedback_MAM_AA_file)
CAM3_wAVD_WV_Feedback_MAM_AA_file.close()

CAM3_wAVD_WV_Feedback_DJF_AA_file = open("CAM3_wAVD_WV_Feedback_DJF_AA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_DJF_AA,CAM3_wAVD_WV_Feedback_DJF_AA_file)
CAM3_wAVD_WV_Feedback_DJF_AA_file.close()

In [39]:
CAM3_wAVD_WV_EnergyB_Annual_GA_file = open("CAM3_wAVD_WV_EnergyB_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_Annual_GA,CAM3_wAVD_WV_EnergyB_Annual_GA_file)
CAM3_wAVD_WV_EnergyB_Annual_GA_file.close()

CAM3_wAVD_WV_EnergyB_SON_GA_file = open("CAM3_wAVD_WV_EnergyB_SON_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_SON_GA,CAM3_wAVD_WV_EnergyB_SON_GA_file)
CAM3_wAVD_WV_EnergyB_SON_GA_file.close()

CAM3_wAVD_WV_EnergyB_JJA_GA_file = open("CAM3_wAVD_WV_EnergyB_JJA_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_JJA_GA,CAM3_wAVD_WV_EnergyB_JJA_GA_file)
CAM3_wAVD_WV_EnergyB_JJA_GA_file.close()

CAM3_wAVD_WV_EnergyB_MAM_GA_file = open("CAM3_wAVD_WV_EnergyB_MAM_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_MAM_GA,CAM3_wAVD_WV_EnergyB_MAM_GA_file)
CAM3_wAVD_WV_EnergyB_MAM_GA_file.close()

CAM3_wAVD_WV_EnergyB_DJF_GA_file = open("CAM3_wAVD_WV_EnergyB_DJF_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_EnergyB_DJF_GA,CAM3_wAVD_WV_EnergyB_DJF_GA_file)
CAM3_wAVD_WV_EnergyB_DJF_GA_file.close()

CAM3_wAVD_WV_Feedback_Annual_GA_file = open("CAM3_wAVD_WV_Feedback_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_Annual_GA,CAM3_wAVD_WV_Feedback_Annual_GA_file)
CAM3_wAVD_WV_Feedback_Annual_GA_file.close()

CAM3_wAVD_WV_Feedback_SON_GA_file = open("CAM3_wAVD_WV_Feedback_SON_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_SON_GA,CAM3_wAVD_WV_Feedback_SON_GA_file)
CAM3_wAVD_WV_Feedback_SON_GA_file.close()

CAM3_wAVD_WV_Feedback_JJA_GA_file = open("CAM3_wAVD_WV_Feedback_JJA_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_JJA_GA,CAM3_wAVD_WV_Feedback_JJA_GA_file)
CAM3_wAVD_WV_Feedback_JJA_GA_file.close()

CAM3_wAVD_WV_Feedback_MAM_GA_file = open("CAM3_wAVD_WV_Feedback_MAM_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_MAM_GA,CAM3_wAVD_WV_Feedback_MAM_GA_file)
CAM3_wAVD_WV_Feedback_MAM_GA_file.close()

CAM3_wAVD_WV_Feedback_DJF_GA_file = open("CAM3_wAVD_WV_Feedback_DJF_GA.pickle","wb")
pickle.dump(CAM3_wAVD_WV_Feedback_DJF_GA,CAM3_wAVD_WV_Feedback_DJF_GA_file)
CAM3_wAVD_WV_Feedback_DJF_GA_file.close()